In [1]:
import numpy as np
import pandas as pd
import sklearn.feature_extraction.text as skfet
import sklearn.cluster as skc
import sklearn.preprocessing as skpp
from scipy.cluster.hierarchy import fcluster, linkage
import sklearn.decomposition as skd
import sklearn.pipeline as skp
import sklearn.manifold as skmi
import sklearn.ensemble as ske

In [2]:
dat = pd.read_csv('../data/data.csv',index_col=None)

In [3]:
dat.head()

,ProductName,Seller,UsageType,Region,CostUSD
0,GCP Fastly for GCP Marketplace,fastly-public-46a6,D27A-A4CF-6B01:Fastlypublic46a6,(not set),230604.040000
1,"GCP NetApp, Inc. NetApp Cloud Volumes Service",41748,64A1-DB8B-7CDA:41748,us-central1,64513.119782
2,"GCP NetApp, Inc. NetApp Cloud Volumes Service",41748,64A1-DB8B-7CDA:41748,us-west4,62623.550218
3,"GCP NetApp, Inc. NetApp Cloud Volumes Service",41748,009E-7D88-7AE8:41748,us-east4,53447.398344
4,"GCP NetApp, Inc. NetApp Cloud Volumes Service",41748,009E-7D88-7AE8:41748,us-west2,50071.497941


In [4]:
unique_tuples = dat.groupby(['ProductName', 'Seller']).size().reset_index(name='Freq')

In [5]:
vectorizer = skp.Pipeline(
    steps = [
        ('tf-idf',skfet.TfidfVectorizer(ngram_range=(2,8),stop_words='english')),
        ('dense', skpp.FunctionTransformer(lambda x: np.array(x.todense())))
    ]
)

In [6]:
vectors = vectorizer.fit_transform(unique_tuples.apply(lambda x: x['Seller'] + " " + x['ProductName'],axis=1))

In [7]:
clust = skc.AgglomerativeClustering(60,linkage='average')

In [8]:
clust.fit(vectors)

AgglomerativeClustering(linkage='average', n_clusters=60)

In [9]:
unique_tuples['labels_'] = clust.labels_

In [14]:
canonical_index = {}

for s in unique_tuples.Seller.unique():
    tmp = unique_tuples[unique_tuples.Seller == s].copy()
    grp = tmp.groupby('labels_').size().reset_index(name='Freq2').sort_values('Freq2',ascending=False)
    canonical_index[s] = grp.labels_.values[0]

In [22]:
ret = pd.DataFrame.from_dict(canonical_index,orient='index')
ret.columns = ['numeric_index']
ret.to_csv('my_map.csv')

In [23]:
## Generate an actual canonical name

canon = {}
for k, v in canonical_index.items():
    if v not in canon:
        canon[v] = []
    canon[v].append(k)

In [25]:
canon_names = {}
for k in canon.keys():
    canon_names[k] = canon[k][0]

In [28]:
canon_names_df = pd.DataFrame.from_dict(canon_names,orient='index').reset_index()
canon_names_df.columns = ['label_','seller']
canon_names_df.to_csv('my_canon.csv')

In [29]:
def get_mapping(fname='my_map.csv'):
    return pd.read_csv(fname)

def get_data(fname='data.csv'):
    return pd.read_csv(fname)

def get_canon(fname='my_canon.csv'):
    return pd.read_csv(fname)

In [31]:
my_map = get_mapping()
data = get_data('../data/data.csv')
my_canon = get_canon()

In [36]:
my_map.columns = ['Seller','label_']

In [37]:
data = pd.merge(data,my_map,on='Seller')

In [40]:
my_canon.columns = ['idx','label_','CanonSeller']
data = pd.merge(data,my_canon,on='label_')

In [43]:
data.groupby('CanonSeller').agg({'CostUSD': 'sum'}).sort_values('CostUSD',ascending=False)

,CostUSD
CanonSeller,
netapp-cloudmanager,392143.115395
fastly-public-46a6,230604.040000
Fortinet Inc.,51222.208493
matillion-etl-for-bq-public,19117.822320
"Palo Alto Networks, Inc.",16946.759129
AlmaLinux,15850.485262
Elasticsearch,9549.286500
Redis Labs,9403.852000
Aiven,5091.731000
